In [1]:
# Main imports
import pandas as pd
import numpy as np

# This method will need to be changed to non local method of accessing data
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import glob
import numpy as np
import os
import pandas as pd
import pandas_profiling

In [0]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [0]:
abb_dir = os.listdir('gdrive/My Drive/airbnb')

In [0]:
data =  pd.read_csv('gdrive/My Drive/airbnb/listings_summary.csv')

In [0]:
scale_mapper2 = {'Private room': 1,
                 'Entire home/apt': 2,
                 'Shared room': 3}

scale_mapper = {'Real Bed': 1,
                'Pull-out Sofa':2,
                'Futon': 3,
                'Couch': 4,
                'Airbed': 5}                 
    

In [0]:
data['bed_type'] = data['bed_type'].replace(scale_mapper)   
data['room_type'] = data['room_type'].replace(scale_mapper2)
      

In [8]:
# Display first 5 observations to ensure proper creation of DataFrame
data.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,...,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2015,https://www.airbnb.com/rooms/2015,20181107122246,2018-11-07,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,none,It is located in the former East Berlin area o...,"This is my home, not a hotel. I rent out occas...","Close to U-Bahn U8 and U2 (metro), Trams M12, ...","Simple kitchen/cooking, refrigerator, microwav...",Always available,"No parties No events No pets No smoking, not e...",nan,nan,https://a0.muscache.com/im/pictures/260fd609-7...,nan,2217,https://www.airbnb.com/users/show/2217,Ian,2008-08-18,"Key Biscayne, Florida, United States",Believe in sharing economy.,within an hour,96%,nan,t,https://a0.muscache.com/im/pictures/21428a22-4...,https://a0.muscache.com/im/pictures/21428a22-4...,Mitte,4.00,4.00,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Berlin, Berlin, Germany",Mitte,Brunnenstr. Süd,...,2.00,1,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil...",nan,$60.00,NaN,NaN,$200.00,$30.00,1,$28.00,4,1125,3 months ago,t,0,21,51,141,2018-11-07,118,2016-04-11,2018-10-28,93.00,10.00,9.00,10.00,10.00,10.00,9.00,t,NaN,nan,f,f,strict_14_with_grace_period,f,f,4,3.76
1,2695,https://www.airbnb.com/rooms/2695,20181107122246,2018-11-07,Prenzlauer Berg close to Mauerpark,NaN,In the summertime we are spending most of our ...,In the summertime we are spending most of our ...,none,NaN,NaN,Within walking distance you'll find the S-Bahn...,Außer deinem Zimmer kannst du noch die Küche u...,NaN,NaN,nan,nan,https://a0.muscache.com/im/pictures/97d4f6e8-1...,nan,2986,https://www.airbnb.com/users/show/2986,Michael,2008-09-16,"Berlin, Berlin, Germany",Living with my wife in Berlin. Like travellin...,NaN,NaN,nan,f,https://a0.muscache.com/im/pictures/50434955-1...,https://a0.muscache.com/im/pictures/50434955-1...,Prenzlauer Berg,1.00,1.00,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Berlin, Berlin, Germany",NaN,Prenzlauer Berg Nordwest,...,1.00,1,"{Wifi,Kitchen,Elevator,Heating,Washer,Essentia...",nan,$17.00,NaN,NaN,$0.00,$0.00,1,$0.00,2,40,7 weeks ago,t,0,0,0,0,2018-11-07,6,2018-07-04,2018-10-01,100.00,10.00,10.00,10.00,10.00,10.00,10.00,t,NaN,nan,f,f,flexible,f,f,1,1.42
2,3176,https://www.airbnb.com/rooms/3176,20181107122246,2018-11-07,Fabulous Flat in great Location,This beautiful first floor apartment is situa...,1st floor (68m2) apartment on Kollwitzplatz/ P...,This beautiful first floor apartment is situa...,none,The neighbourhood is famous for its variety of...,We welcome FAMILIES and cater especially for y...,"We are 5 min walk away from the tram M2, whic...",The apartment will be entirely yours. We are c...,Feel free to a

In [9]:
# View column names
data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

# Gather the right data

The model will only be exposed to certain data. For this reason, it is on necessary to train the model on certain types of
data from what is available. This CSV contains all of that data, so I will extract only the needed columns.

In [0]:
# Define the needed columns
columns = [
    'minimum_nights', 'bedrooms', 'bathrooms', 'accommodates', 
    'bed_type', 'security_deposit', 'room_type', 'zipcode', 
    'amenities', 'price'
]

# Select the subset from data
data_2 = data[columns]

In [11]:
# Check that the DataFrame has been altered in the expected way
data_2.head()

,minimum_nights,bedrooms,bathrooms,accommodates,bed_type,security_deposit,room_type,zipcode,amenities,price
0,4,1.00,1.00,3,1,$200.00,2,10119,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil...",$60.00
1,2,1.00,1.00,2,1,$0.00,1,10437,"{Wifi,Kitchen,Elevator,Heating,Washer,Essentia...",$17.00
2,62,1.00,1.00,4,1,$200.00,2,10405,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc...",$90.00
3,5,1.00,1.00,2,2,$250.00,1,10777,"{Internet,Wifi,""Pets allowed"",""Pets live on th...",$26.00
4,2,1.00,1.00,2,1,$0.00,1,10437,"{Wifi,Heating,""Family/kid friendly"",Essentials...",$42.00


In [12]:
# Check dtypes on each column
for column in data_2.columns:
    print(
        f"{column}:\t{data_2[column].dtype}"
    )

minimum_nights:	int64
bedrooms:	float64
bathrooms:	float64
accommodates:	int64
bed_type:	int64
security_deposit:	object
room_type:	int64
zipcode:	object
amenities:	object
price:	object


# Get the data in the right format

Some of the data that should be in a numeric data type is currently a string. The amenities looks like it was poorly converted
from JSON format. The data will need to be cleaned before we can use it to make any predictions.

In [0]:
# Create a function to clean the security deposit column
def clean_security_deposit(value):
    """
    Function accepts single value inputs. If the value is a string, 
    it will strip the '$' and ',' out of the string before converting
    the string to an integer.
    """
    try:
        if type(value) != str:
            return value
        else:
            new_value = value.replace("$", "")
            new_value_2 = new_value.replace(",","")
            return int(float(new_value_2))
    except:
        new_value = str(value).replace("$", "")
        new_value_2 = new_value.replace(",", "")
        return int(float(new_value_2))
    


In [0]:
# Create a function to clean the zipcode column
def clean_zipcode(value):
    """
    Function accepts single value inputs. If the regex pattern is matched, 
    return the match. 
    """
    if type(value) != str:
        return value
    else:
        pattern = '\d+'
        result = re.findall(pattern, value)
        return int(result[0])


In [0]:
# Create a function to clean the price column
def clean_price(value):
    """
    Function accepts single value inputs. If the is a string,
    it will strip the '$' and ',' out of the string before converting
    the string to a float.
    """
    try:
        if type(value) != str:
            return value
        else:
            new_value = value.replace("$", "")
            new_value_2 = new_value.replace(",", "")
            return float(new_value_2)
    except:
        new_value = str(value).replace("$", "")
        new_value_2 = new_value.replace(",", "")
        return float(new_value_2)
    

In [0]:
# Create function to pull needed amenities out into a list
def fix_amenities(value):
    """
    Function accepts single value inputs. If the regex pattern is matched, 
    it will return a list of all values that match and are represented as
    an amenity that we want.
    """
    pattern = '\w+\s?\w+\s?\w+\s?\w+|\w+\s?\w+\s?\w+|\w+\s?\w+|\w+'
    result = re.findall(pattern, value.lower())
    return result


In [0]:
# Import re for regex matching
import re


# Apply functions to DataFrame columns
data_3 = data_2.copy()
data_3['security_deposit'] = data_2['security_deposit'].apply(clean_security_deposit)
data_3['zipcode'] = data_2['zipcode'].apply(clean_zipcode)
data_3['price'] = data_2['price'].apply(clean_price)

# Fix amenities to work with function
data_3['amenities'] = data_2['amenities'].apply(fix_amenities)


In [0]:
# OneHot encode amenities
data_3 = data_3.drop('amenities', 1).join(data_3['amenities'].str.join('|').str.get_dummies())
# Remove nans, unable to impute accurately
data_final = data_3.dropna()

In [19]:
data_final.head()

,minimum_nights,bedrooms,bathrooms,accommodates,bed_type,security_deposit,room_type,zipcode,price,24,accessible,air conditioning,air purifier,baby bath,baby monitor,babysitter recommendations,bath towel,bathroom essentials,bathtub,bathtub with bath chair,bbq grill,beach essentials,beachfront,bed linens,bedroom comforts,body soap,breakfast,breakfast table,building staff,buzzer,cable tv,carbon monoxide detector,cat,ceiling hoist,changing table,children,cleaning before checkout,coffee maker,convection oven,cooking basics,...,refrigerator,roll,room,s,s books and toys,s dinnerware,safety card,self check,shampoo,shower,shower chair,single level home,ski,smart lock,smart tv,smoke detector,smoking allowed,sound system,stair gates,step,stove,suitable for events,table corner guards,toilet,toilet paper,translation missing,travel crib,tv,washer,waterfront,well,wheelchair accessible,wide clearance to bed,wide clearance to shower,wide doorway,wide entryway,wide hallway clearance,wifi,window guards,wireless intercom
0,4,1.00,1.00,3,1,200.00,2,10119.00,60.00,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1,2,1.00,1.00,2,1,0.00,1,10437.00,17.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2,62,1.00,1.00,4,1,200.00,2,10405.00,90.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
3,5,1.00,1.00,2,2,250.00,1,10777.00,26.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1
4,2,1.00,1.00,2,1,0.00,1,10437.00,42.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [20]:
data_3.head()

,minimum_nights,bedrooms,bathrooms,accommodates,bed_type,security_deposit,room_type,zipcode,price,24,accessible,air conditioning,air purifier,baby bath,baby monitor,babysitter recommendations,bath towel,bathroom essentials,bathtub,bathtub with bath chair,bbq grill,beach essentials,beachfront,bed linens,bedroom comforts,body soap,breakfast,breakfast table,building staff,buzzer,cable tv,carbon monoxide detector,cat,ceiling hoist,changing table,children,cleaning before checkout,coffee maker,convection oven,cooking basics,...,refrigerator,roll,room,s,s books and toys,s dinnerware,safety card,self check,shampoo,shower,shower chair,single level home,ski,smart lock,smart tv,smoke detector,smoking allowed,sound system,stair gates,step,stove,suitable for events,table corner guards,toilet,toilet paper,translation missing,travel crib,tv,washer,waterfront,well,wheelchair accessible,wide clearance to bed,wide clearance to shower,wide doorway,wide entryway,wide hallway clearance,wifi,window guards,wireless intercom
0,4,1.00,1.00,3,1,200.00,2,10119.00,60.00,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1,2,1.00,1.00,2,1,0.00,1,10437.00,17.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2,62,1.00,1.00,4,1,200.00,2,10405.00,90.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
3,5,1.00,1.00,2,2,250.00,1,10777.00,26.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1
4,2,1.00,1.00,2,1,0.00,1,10437.00,42.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [21]:
!pip install category_encoders

In [0]:
# Import sklearn modules
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder, OneHotEncoder


In [0]:
# Train / Test split
# Split into feature matrix and target vector


target = 'price'

X = data_final.drop(columns=target)
y = data_final[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [24]:
X_train['laundry'] = np.zeros(len(X_train))
X_train['security'] = np.zeros(len(X_train))
X_train['patio'] = np.zeros(len(X_train))
X_train['spa'] = np.zeros(len(X_train))
X_train['cleaning'] = np.zeros(len(X_train))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [25]:
X_train['laundry'] = X_train['laundry'].astype(int)
X_train['security'] = X_train['security'].astype(int)
X_train['patio'] = X_train['patio'].astype(int)
X_train['spa'] = X_train['spa'].astype(int)
X_train['cleaning'] = X_train['cleaning'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [26]:
X_test['laundry'] = np.zeros(len(X_test))
X_test['security'] = np.zeros(len(X_test))
X_test['patio'] = np.zeros(len(X_test))
X_test['spa'] = np.zeros(len(X_test))
X_test['cleaning'] = np.zeros(len(X_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [27]:
X_test['laundry'] = X_test['laundry'].astype(int)
X_test['security'] = X_test['security'].astype(int)
X_test['patio'] = X_test['patio'].astype(int)
X_test['spa'] = X_test['spa'].astype(int)
X_test['cleaning'] = X_test['cleaning'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [28]:
# Create Pipeline
pipeline = Pipeline(
    [('encoder', OrdinalEncoder()),
     ('model', LinearRegression())]
)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('encoder',
                 OrdinalEncoder(cols=[], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[], return_df=True, verbose=0)),
                ('model',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [0]:
y_pred = pipeline.predict(X_train)

In [30]:
mean_squared_error(y_train, y_pred)

61222.268338448404

# Not great, but it's something

The results aren't amazing, actually fairly far from it. But at least this is giving us _something_

In [0]:
# Lets create a new set of data subset from only the highest review scores
new_data = data[data['review_scores_value']>=9]

In [0]:
# Pull out only the columns we care about
subset_data = new_data[columns]

In [0]:
# Impute 0 for security deposit if NaN
df = subset_data.copy()
df['security_deposit'] = df['security_deposit'].fillna(0)

In [0]:
# Drop other NaN values, they can't be imputed accurately
df = df.dropna()

In [0]:
# Clean the data
df['security_deposit'] = df['security_deposit'].apply(clean_security_deposit)
df['price'] = df['price'].apply(clean_price)
df['zipcode'] = df['zipcode'].apply(clean_zipcode)
df['amenities'] = df['amenities'].apply(fix_amenities)

In [36]:
df

,minimum_nights,bedrooms,bathrooms,accommodates,bed_type,security_deposit,room_type,zipcode,amenities,price
0,4,1.00,1.00,3,1,200,2,10119,"[tv, cable tv, wifi, kitchen, gym, heating, fa...",60.00
1,2,1.00,1.00,2,1,0,1,10437,"[wifi, kitchen, elevator, heating, washer, ess...",17.00
2,62,1.00,1.00,4,1,200,2,10405,"[internet, wifi, kitchen, buzzer, wireless int...",90.00
3,5,1.00,1.00,2,2,250,1,10777,"[internet, wifi, pets allowed, pets live on th...",26.00
4,2,1.00,1.00,2,1,0,1,10437,"[wifi, heating, family, kid friendly, essentia...",42.00
...,...,...,...,...,...,...,...,...,...,...
22346,1,1.00,1.00,1,1,0,3,10115,"[wifi, heating, washer, first aid kit, safety ...",18.00
22352,1,1.00,1.00,4,1,600,1,13359,"[wifi, air conditioning, kitchen, smoking allo...",40.00
22367,1,1.00,1.00,2,1,0,1,10247,"[wifi, kitchen, breakfast, heating, washer, fi...",50.00
22398,2,2.00,1.00,4,1,0,2,10557,"[tv, wifi, air conditioning, kitchen, free par...",45.00


In [0]:
# OneHot amenities
df = df.join(df['amenities'].str.join('|').str.get_dummies())

In [0]:
# Drop amenities column
df = df.drop('amenities', 1)

In [0]:
# Re-sort amenities by occurance
amenity_count = dict()

for column in df:
    if column in columns:
        pass
    else:
        amenity_count[column] = df[column].value_counts().to_dict()
        
# Set to dataframe to sort
df_amenities = pd.DataFrame(amenity_count)

# Create list of popular index
amenities = df_amenities.iloc[1].sort_values(ascending=False)[:52].index.to_list()

# Remove nonsense indicies
amenities.remove('en')
amenities.remove('in')
amenities.remove('translation missing')
amenities.remove('hosting_amenity_50')
amenities.remove('hosting_amenity_49')


In [0]:
# Feed new data through the pipeline

# Split into feature matrix / target vector
new_columns = [
    'minimum_nights', 'bedrooms', 'bathrooms', 'accommodates', 
    'bed_type', 'security_deposit', 'room_type', 'zipcode', 
    'price'
]

X = df[new_columns + amenities].drop(columns=target)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [0]:
# Fit ridge regressor as baseline model
from sklearn.linear_model import Ridge
model = Pipeline(
    [('encoder', OrdinalEncoder()), 
     ('reg', Ridge(alpha=10, solver='auto', tol=0.05))]
)


In [42]:
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('encoder',
                 OrdinalEncoder(cols=[], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[], return_df=True, verbose=0)),
                ('reg',
                 Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
                       normalize=False, random_state=None, solver='auto',
                       tol=0.05))],
         verbose=False)

In [43]:
model.score(X_train, y_train) # r^2 is 8%

0.08110959807915175

In [0]:
y_pred = model.predict(X_test)

In [45]:
y_pred[:5]

array([86.61193121, 47.79530451, 16.47172705, 80.0104625 , 94.52625868])

In [46]:
y_test[:5]

11300   60.00
14162   40.00
9454    25.00
16880   27.00
1404    75.00
Name: price, dtype: float64

In [0]:
import matplotlib.pyplot as plt
plt.scatter(y_pred, y_test)
plt.show()

In [0]:
# Create truncation function -> min 15
y_pred_2 = [15.0 if x < 15.0 else round(x) for x in y_pred]

In [49]:
# Calc R^2 score for truncated predictions
from sklearn.metrics import r2_score


r2_score(y_test, y_pred_2)

0.20528258819205647

# An improvement

This model scores 20% r-squared, which is an improvement over the original baseline model.

In [0]:
# Save the model object
import pickle

file_model = open('model.obj', 'wb')
pickle.dump(model, file_model)

In [0]:
file_model.close()

In [52]:
X_train.head()

,minimum_nights,bedrooms,bathrooms,accommodates,bed_type,security_deposit,room_type,zipcode,wifi,heating,kitchen,essentials,washer,hair dryer,laptop friendly workspace,hangers,iron,hot water,shampoo,tv,family,kid friendly,internet,host greets you,smoke detector,buzzer,wireless intercom,free street parking,refrigerator,dishes and silverware,bed linens,cooking basics,stove,lock on bedroom door,oven,elevator,coffee maker,smoking allowed,first aid kit,cable tv,dishwasher,long term stays allowed,luggage dropoff allowed,dryer,fire extinguisher,pets allowed,extra pillows and blankets,patio or balcony,microwave,private entrance,paid parking off premises,safety card,free parking on premises,private living room,bathtub
4730,1,1.00,1.00,2,1,0,1,12051,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13957,10,1.00,1.00,4,1,500,2,10829,1,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
18166,2,1.00,1.00,1,1,0,1,12163,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,0,1,1,1,1,1,0,1,0,1,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
14578,1,1.00,1.00,2,1,0,1,10785,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4655,4,1.00,1.00,2,1,200,2,10247,1,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
X_train.bed_type.value_counts()

1    13053
2      278
3      140
4       42
5       11
Name: bed_type, dtype: int64

In [54]:
X_train.room_type.value_counts()

1    6993
2    6382
3     149
Name: room_type, dtype: int64

In [55]:
X_train.head()

,minimum_nights,bedrooms,bathrooms,accommodates,bed_type,security_deposit,room_type,zipcode,wifi,heating,kitchen,essentials,washer,hair dryer,laptop friendly workspace,hangers,iron,hot water,shampoo,tv,family,kid friendly,internet,host greets you,smoke detector,buzzer,wireless intercom,free street parking,refrigerator,dishes and silverware,bed linens,cooking basics,stove,lock on bedroom door,oven,elevator,coffee maker,smoking allowed,first aid kit,cable tv,dishwasher,long term stays allowed,luggage dropoff allowed,dryer,fire extinguisher,pets allowed,extra pillows and blankets,patio or balcony,microwave,private entrance,paid parking off premises,safety card,free parking on premises,private living room,bathtub
4730,1,1.00,1.00,2,1,0,1,12051,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13957,10,1.00,1.00,4,1,500,2,10829,1,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
18166,2,1.00,1.00,1,1,0,1,12163,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,0,1,1,1,1,1,0,1,0,1,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
14578,1,1.00,1.00,2,1,0,1,10785,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4655,4,1.00,1.00,2,1,200,2,10247,1,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder

In [0]:
# Have to ordinal encode bed_type, room_type 


In [57]:
X_train.head()

,minimum_nights,bedrooms,bathrooms,accommodates,bed_type,security_deposit,room_type,zipcode,wifi,heating,kitchen,essentials,washer,hair dryer,laptop friendly workspace,hangers,iron,hot water,shampoo,tv,family,kid friendly,internet,host greets you,smoke detector,buzzer,wireless intercom,free street parking,refrigerator,dishes and silverware,bed linens,cooking basics,stove,lock on bedroom door,oven,elevator,coffee maker,smoking allowed,first aid kit,cable tv,dishwasher,long term stays allowed,luggage dropoff allowed,dryer,fire extinguisher,pets allowed,extra pillows and blankets,patio or balcony,microwave,private entrance,paid parking off premises,safety card,free parking on premises,private living room,bathtub
4730,1,1.00,1.00,2,1,0,1,12051,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13957,10,1.00,1.00,4,1,500,2,10829,1,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
18166,2,1.00,1.00,1,1,0,1,12163,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,0,1,1,1,1,1,0,1,0,1,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
14578,1,1.00,1.00,2,1,0,1,10785,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4655,4,1.00,1.00,2,1,200,2,10247,1,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
model = XGBClassifier()
model.fit(X_train, y_train)

In [60]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print('Accuracy: %.2f%%' % (accuracy * 100.0))

Accuracy: 7.75%


4730    40.00
13957   55.00
18166   29.00
14578   35.00
4655    45.00
Name: price, dtype: float64

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [65]:
from tensorflow.keras.models import Sequential, Model

X_train.head()

,minimum_nights,bedrooms,bathrooms,accommodates,bed_type,security_deposit,room_type,zipcode,wifi,heating,kitchen,essentials,washer,hair dryer,laptop friendly workspace,hangers,iron,hot water,shampoo,tv,family,kid friendly,internet,host greets you,smoke detector,buzzer,wireless intercom,free street parking,refrigerator,dishes and silverware,bed linens,cooking basics,stove,lock on bedroom door,oven,elevator,coffee maker,smoking allowed,first aid kit,cable tv,dishwasher,long term stays allowed,luggage dropoff allowed,dryer,fire extinguisher,pets allowed,extra pillows and blankets,patio or balcony,microwave,private entrance,paid parking off premises,safety card,free parking on premises,private living room,bathtub
4730,1,1.00,1.00,2,1,0,1,12051,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13957,10,1.00,1.00,4,1,500,2,10829,1,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
18166,2,1.00,1.00,1,1,0,1,12163,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,0,1,1,1,1,1,0,1,0,1,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
14578,1,1.00,1.00,2,1,0,1,10785,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4655,4,1.00,1.00,2,1,200,2,10247,1,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [67]:
X_train.shape

(13524, 55)

In [70]:
model = tf.keras.Sequential()
model.add(layers.Dense(64, input_shape=(55,), activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='relu'))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
from statistics import mean

In [0]:
model.compile(optimizer='adam', loss='mse')

In [77]:
model.fit(X_train,y_train,epochs=50,batch_size=128)

Train on 13524 samples
Epoch 1/50
13524/13524 [==============================] - 1s 51us/sample - loss: 24844.0419
Epoch 2/50
13524/13524 [==============================] - 0s 20us/sample - loss: 23595.1819
Epoch 3/50
13524/13524 [==============================] - 0s 17us/sample - loss: 23595.1832
Epoch 4/50
13524/13524 [==============================] - 0s 18us/sample - loss: 23595.1829
Epoch 5/50
13524/13524 [==============================] - 0s 20us/sample - loss: 23595.1826
Epoch 6/50
13524/13524 [==============================] - 0s 19us/sample - loss: 23595.1826
Epoch 7/50
13524/13524 [==============================] - 0s 18us/sample - loss: 23595.1823
Epoch 8/50
13524/13524 [==============================] - 0s 18us/sample - loss: 23595.1816
Epoch 9/50
13524/13524 [==============================] - 0s 17us/sample - loss: 23595.1833
Epoch 10/50
13524/13524 [==============================] - 0s 18us/sample - loss: 23595.1827
Epoch 11/50
13524/13524 [==============================]